## importing 

In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import requests
import os
import shutil
from shapely.geometry import Point, LineString, MultiPoint
import matplotlib
from ipywidgets import interact
import seaborn as sns
import math
import stats
import scipy
from dotenv import load_dotenv, find_dotenv
from urllib.parse import urljoin
import requests
from io import StringIO, BytesIO
import osmnx as ox
import time
import shapely
from folium import Map, CircleMarker, Vega, Popup, Marker, PolyLine, Icon, Choropleth, LayerControl
from folium.plugins import MarkerCluster, HeatMap, BeautifyIcon
from folium.map import LayerControl, Layer, FeatureGroup
from folium.features import ColorLine, GeoJsonPopup
import json
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

## testing remote opening of files

In [2]:
# constructor of google download links
dl_construct = 'https://drive.google.com/uc?export=download&id='
# id from share link google drive
file_id = '1zUzr93toblHkH2f3Zj5CEuD81qND7WYd'
csv = requests.get(f'{dl_construct}{file_id}').content
memory = BytesIO(csv)
df = pd.read_csv(memory)
df.shape

(8669, 110)

In [3]:
list(df.columns)

['Unnamed: 0',
 'Link_ID',
 'linkid',
 'Daily_Average_Traffic_Intensity',
 'Average_Velocity_of_Vehicle_Traffic',
 'Median_of_velocity_of_Vehicle_Traffic',
 'FirstQuartil_of_velocity_of_Vehicle_Traffic',
 'ThirdQuartil_of_velocity_of_Vehicle_Traffic',
 'Func_Class',
 'Speed_Cat',
 'geometry',
 'feat_barrier_block',
 'feat_barrier_bollard',
 'feat_barrier_border_control',
 'feat_barrier_bump_gate',
 'feat_barrier_cattle_grid',
 'feat_barrier_chain',
 'feat_barrier_cycle_barrier',
 'feat_barrier_debris',
 'feat_barrier_door',
 'feat_barrier_entrance',
 'feat_barrier_fence',
 'feat_barrier_gate',
 'feat_barrier_height_restrictor',
 'feat_barrier_jersey_barrier',
 'feat_barrier_kerb',
 'feat_barrier_kissing_gate',
 'feat_barrier_lift_gate',
 'feat_barrier_rope',
 'feat_barrier_sally_port',
 'feat_barrier_security_check',
 'feat_barrier_stile',
 'feat_barrier_swing_gate',
 'feat_barrier_toll_booth',
 'feat_barrier_toll_gantry',
 'feat_barrier_turnstile',
 'feat_barrier_wall',
 'feat_barrier

In [4]:
dfd = df.drop(columns=['Link_ID','Unnamed: 0','Daily_Average_Traffic_Intensity','Average_Velocity_of_Vehicle_Traffic',
       'Median_of_velocity_of_Vehicle_Traffic',
       'FirstQuartil_of_velocity_of_Vehicle_Traffic',
       'ThirdQuartil_of_velocity_of_Vehicle_Traffic', 'Func_Class',
       'Speed_Cat','geometry', 'points'])

In [5]:
def scaling_numerical(data):
    numerical = data.columns.drop('linkid')
    scaler = MinMaxScaler()
    data_scaled = data.copy()
    for column in numerical:
        scaler.fit(data_scaled[[column]])
        data_scaled[column]=scaler.transform(data_scaled[[column]]) 
    return data_scaled
dfd = scaling_numerical(dfd)

In [6]:
dfd.head(10)

,linkid,feat_barrier_block,feat_barrier_bollard,feat_barrier_border_control,feat_barrier_bump_gate,feat_barrier_cattle_grid,feat_barrier_chain,feat_barrier_cycle_barrier,feat_barrier_debris,feat_barrier_door,...,feat_man_made_water_tap,feat_man_made_water_tower,feat_man_made_water_well,feat_man_made_water_works,feat_man_made_watermill,feat_man_made_windmill,feat_man_made_windpump,feat_man_made_works,feat_man_made_yes,sum_points
0,80216819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667
1,80216858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.177083
2,80216859,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667
3,80216883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333
4,80216931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.093750
5,80217095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833
6,80217096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,80217097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031250
8,80217147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,80217198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
